# Packages

In [ ]:
import qcodes as qc
from qcodes.logger.logger import start_all_logging
import datetime
import numpy as np
%matplotlib notebook
from datetime import date,datetime
import pyvisa
import sys
sys.path.append('') # Write the directory containing ZVL13_VNA.py
import json
from ZVL_VNA import ZVL

# Instrument and station initialization

In [ ]:
start_all_logging()


#Instruments
VNA = ZVL('ZVL', 'TCPIP0::vnarstafuri.fisica.unina.it::inst0::INSTR')


#Station inizialization
station = qc.Station()
station.add_component(VNA)

# Parameters definition

In [ ]:
#########################
##Parameters definition##
#########################

parameter_snap={}

S_parameter = 'S11'
meas_format = 'dbm'
start_freq = 4
end_freq = 8
#center_freq = 5
#span_freq = 1
VNA_power = -20
IF_bandwidth = 1000
Averages = 4
points_VNA = 101 #Max:4001



VNA.npts(points_VNA)
VNA.S_parameter(S_parameter)
VNA.format(meas_format)
VNA.bandwidth(IF_bandwidth)
VNA.power(VNA_power)
VNA.avg(Averages)
VNA.start(start_freq*1e9)
VNA.stop(end_freq*1e9)
#VNA.center(center_freq*1e9)
#VNA.span(span_freq*1e9)


# Trace acquisition in Network Analyzer Mode

In [ ]:
#########################
#######Mode setting######
#########################

Mode = VNA.mode() 

if Mode == 'SAN':
    VNA.mode('nwa')

#########################
##Parameters definition##
#########################

S_parameter = 'S11'
meas_format = 'dbm'
start_freq = 4 #GHz
end_freq = 8 #GHz
#center_freq = 5 #GHz
#span_freq = 1 #GHz
VNA_power = -20 #dBm
IF_bandwidth = 1000 #Hz
Averages = 4
points_VNA = 101 #Max:4001


#Parameters setting in the VNA
VNA.calibration() #Switches on the most recent calibration in the VNA 
VNA.S_parameter(S_parameter)
VNA.npts(points_VNA)
VNA.format(meas_format)
VNA.bandwidth(IF_bandwidth)
VNA.power(VNA_power)
VNA.avg(Averages)
VNA.start(start_freq*1e9)
VNA.stop(end_freq*1e9)
#VNA.center(center_freq*1e9)
#VNA.span(span_freq*1e9)

#VNA Timeout setting
original_timeout = VNA.timeout()
new_timeout = 10
VNA.timeout(new_timeout)

#############################
##Experiment initialization##
#############################

#Experiment definition
exp_name = 'exp_name'
sample = 'sample'
exp=qc.load_or_create_experiment(experiment_name=exp_name,
                          sample_name=sample)



#Parameters registration
meas = qc.Measurement(exp=exp, station=station) 
meas.register_parameter(VNA.trace) 
meas.register_parameter(VNA.S_trace)

#Start measurement
VNA.rf_power(1) #Switches on the power
VNA.cont_meas_on() #Switches on the sweep
VNA.electrical_delay_auto() #Removes the electrical delay automatically


with meas.run() as datasaver:
    VNA.autoscale() #Autoscale the trace on the VNA display
    get_v = VNA.trace.get() #Get the trace displayed on the VNA
    get_v2 = VNA.S_trace.get() #Get the complex form of the scattering parameter under test
    datasaver.add_result((VNA.trace, get_v))
    datasaver.add_result((VNA.S_trace, get_v2))


#Shutdown devices
VNA.rf_power(0)
VNA.cont_meas_off()

###############
##Data saving##
###############

captured_run_id=datasaver.dataset.run_id
dataset = qc.load_by_run_spec(captured_run_id=captured_run_id)
freq = dataset.get_parameter_data('ZVL_ZVL_frequency')['ZVL_ZVL_frequency']['ZVL_ZVL_frequency']
save_trace = dataset.get_parameter_data('ZVL_trace')['ZVL_trace']['ZVL_trace']
S_trace = dataset.get_parameter_data('ZVL_S_trace')['ZVL_S_trace']['ZVL_S_trace']

## Plot

In [ ]:
plt.figure(figsize=(14*0.8, 10*0.8), dpi= 80) #facecolor='w', edgecolor='k'

plt.plot(freq[0]*1e-9,save_trace[0])

plt.rc('axes', labelsize=12)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12*1.2)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12*1.2)  

x_label=''
y_label=''
plt.xlabel(x_label, size=25)
plt.ylabel(y_label, size=25)
plt.title(plot_title, size=18)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)

# Trace acquisition in Spectrum Analyzer Mode

In [ ]:
# #######Mode setting######
#########################

Mode = VNA.mode() 

if Mode == 'NWA':
    VNA.mode('sa')

#########################
##Parameters definition##
#########################


start_freq = 4 #GHz
end_freq = 8 #GHz
#center_freq = 5 #GHz
#span_freq = 1 #GHz
Averages = 4
points_VNA = 101 #Max:4001


#Parameters setting in the VNA
VNA.calibration() #Switches on the most recent calibration in the VNA 
VNA.S_parameter(S_parameter)
VNA.npts(points_VNA)
VNA.format(meas_format)
VNA.bandwidth(IF_bandwidth)
VNA.power(VNA_power)
VNA.avg(Averages)
VNA.start(start_freq*1e9)
VNA.stop(end_freq*1e9)
#VNA.center(center_freq*1e9)
#VNA.span(span_freq*1e9)

#VNA Timeout setting
original_timeout = VNA.timeout()
new_timeout = 10
VNA.timeout(new_timeout)

#############################
##Experiment initialization##
#############################

#Experiment definition
exp_name = 'exp_name'
sample = 'sample'
exp=qc.load_or_create_experiment(experiment_name=exp_name,
                          sample_name=sample)



#Parameters registration
meas = qc.Measurement(exp=exp, station=station) 
meas.register_parameter(VNA.spectrum) 

#Start measurement
VNA.cont_meas_on() #Switches on the sweep


with meas.run() as datasaver:
    VNA.autoscale() #Autoscale the trace on the VNA display
    get_v = VNA.spectrum.get() #Get the spectrum seen by Port 2
    datasaver.add_result((VNA.spectrum, get_v))


#Shutdown devices
VNA.cont_meas_off()

###############
##Data saving##
###############

captured_run_id=datasaver.dataset.run_id
dataset = qc.load_by_run_spec(captured_run_id=captured_run_id)
freq = dataset.get_parameter_data('ZVL_ZVL_frequency')['ZVL_ZVL_frequency']['ZVL_ZVL_frequency']
save_trace = dataset.get_parameter_data('ZVL_trace')['ZVL_trace']['ZVL_trace']
S_trace = dataset.get_parameter_data('ZVL_S_trace')['ZVL_S_trace']['ZVL_S_trace']

## Plot 

In [ ]:
plt.figure(figsize=(14*0.8, 10*0.8), dpi= 80) 

plt.plot(freq[0]*1e-9,spectrum[0])

plt.rc('axes', labelsize=12)   
plt.rc('xtick', labelsize=12*1.2)    
plt.rc('ytick', labelsize=12*1.2)  

x_label=''
y_label=''
plt.xlabel(x_label, size=25)
plt.ylabel(y_label, size=25)
plt.title(plot_title, size=18)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)